Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

# uncleaned train, valid and test sets
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [2]:
# cleaned train, valid and test sets
pickle_file = 'notMNIST_c.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset_c']
  train_labels = save['train_labels_c']
  valid_dataset = save['valid_dataset_c']
  valid_labels = save['valid_labels_c']
  test_dataset = save['test_dataset_c']
  test_labels = save['test_labels_c']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (145051, 28, 28) (145051,)
Validation set (7338, 28, 28) (7338,)
Test set (6722, 28, 28) (6722,)


# cleaned training, unclean valid and test
pickle_file1 = 'notMNIST_c.pickle'

with open(pickle_file1, 'rb') as f1:
  save = pickle.load(f1)
  train_dataset = save['train_dataset_c']
  train_labels = save['train_labels_c']
  del save
  

pickle_file2 = 'notMNIST.pickle'

with open(pickle_file2, 'rb') as f2:
  save = pickle.load(f2)
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
    
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (145051, 784) (145051, 10)
Validation set (7338, 784) (7338, 10)
Test set (6722, 784) (6722, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000 # was 100000

L2_coef = .01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 801 #801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 48.073830
Training accuracy: 7.6%
Validation accuracy: 10.8%
Loss at step 100: 12.043386
Training accuracy: 68.3%
Validation accuracy: 68.8%
Loss at step 200: 4.593434
Training accuracy: 73.5%
Validation accuracy: 73.7%
Loss at step 300: 2.095817
Training accuracy: 77.0%
Validation accuracy: 76.9%
Loss at step 400: 1.260064
Training accuracy: 78.7%
Validation accuracy: 78.4%
Loss at step 500: 0.976689
Training accuracy: 79.3%
Validation accuracy: 79.0%
Loss at step 600: 0.879272
Training accuracy: 79.5%
Validation accuracy: 79.3%
Loss at step 700: 0.845426
Training accuracy: 79.5%
Validation accuracy: 79.4%
Loss at step 800: 0.833567
Training accuracy: 79.6%
Validation accuracy: 79.4%
Test accuracy: 86.4%


Now for the neural network

In [15]:
batch_size = 128
n_hidden = 1024
L2_coef = .001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases = tf.Variable(tf.zeros([n_hidden]))
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
    
    # Training computation.
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  logits = tf.matmul(relu_layer, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_valid, weights2) + biases2)
  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_test, weights2) + biases2)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 670.485046
Minibatch accuracy: 3.9%
Validation accuracy: 28.6%
Minibatch loss at step 500: 197.721786
Minibatch accuracy: 72.7%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 115.200890
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 1500: 68.775314
Minibatch accuracy: 83.6%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 41.214401
Minibatch accuracy: 86.7%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 25.032356
Minibatch accuracy: 92.2%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 15.480123
Minibatch accuracy: 88.3%
Validation accuracy: 82.9%
Test accuracy: 90.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [28]:
batch_size = 128
n_hidden = 1024
L2_coef = 0 #.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases = tf.Variable(tf.zeros([n_hidden]))
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
    
    # Training computation.
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  logits = tf.matmul(relu_layer, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_valid, weights2) + biases2)
  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_test, weights2) + biases2)

In [29]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    ##offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step % 2) * batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 387.011841
Minibatch accuracy: 13.3%
Validation accuracy: 23.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.2%
Test accuracy: 72.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
batch_size = 128
n_hidden = 1024
L2_coef = 0 #.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases = tf.Variable(tf.zeros([n_hidden]))
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
    
    # Training computation.
  relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  logits = tf.matmul(relu_layer_dropout, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    
  relu_layer_NO_dropout = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  logits_NO_droput = tf.matmul(relu_layer_NO_dropout, weights2) + biases2
  train_prediction = tf.nn.softmax(logits_NO_droput)
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_valid, weights2) + biases2)
  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_test, weights2) + biases2)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    ##offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step % 2) * batch_size # this causes overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 927.082397
Minibatch accuracy: 10.2%
Validation accuracy: 34.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Test accuracy: 76.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [41]:
batch_size = 128
n_hidden = 1024
num_steps = 10001
L2_coef = 0.001 #.001
init_learn_rate = .1
final_learn_rate = 0.005
#learning_rate=0.1

init_mean = 0
standard_dev = .1
bias_init = .01

steps_per_batch = np.ceil(train_dataset.shape[0]/batch_size)

loops_over_data = np.floor(num_steps/steps_per_batch)

# init_learn_rate*x**loops_over_data = 0.01
# x = (.01/init_learn_rate)**(1/loops_over_data)

decay_rate = (final_learn_rate/init_learn_rate)**(1/loops_over_data) # should be about .7 for 10 loops
print(decay_rate)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden],init_mean,standard_dev))
  biases = tf.Variable(tf.zeros([n_hidden]))
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, steps_per_batch, decay_rate, staircase=True)
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  #biases2 = tf.Variable(tf.zeros([num_labels]))
  biases2 = tf.Variable(bias_init * tf.ones([num_labels]))

  
    
    # Training computation.
  #relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  logits = tf.matmul(relu_layer_dropout, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    
  relu_layer_NO_dropout = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  logits_NO_droput = tf.matmul(relu_layer_NO_dropout, weights2) + biases2
  train_prediction = tf.nn.softmax(logits_NO_droput)
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_valid, weights2) + biases2)
  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_test, weights2) + biases2)

0.687656021934


In [42]:

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    ##offset = (step % 2) * batch_size # this causes overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 106.587257
Minibatch accuracy: 3.1%
Validation accuracy: 34.0%
Minibatch loss at step 500: 26.585846
Minibatch accuracy: 78.9%
Validation accuracy: 76.6%
Minibatch loss at step 1000: 17.141062
Minibatch accuracy: 76.6%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 12.211351
Minibatch accuracy: 80.5%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 7.761628
Minibatch accuracy: 78.9%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 5.915490
Minibatch accuracy: 85.9%
Validation accuracy: 80.6%
Minibatch loss at step 3000: 7.125637
Minibatch accuracy: 83.6%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 6.746795
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 4000: 5.755774
Minibatch accuracy: 83.6%
Validation accuracy: 81.2%
Minibatch loss at step 4500: 4.796015
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 5000: 4.462362
Minibatch accuracy: 83.6%
Validation acc

add a layer


In [41]:
batch_size = 256
n_hidden = 700
n_hidden2 = 700
num_steps = 8001
L2_coef = .001
init_learn_rate = 0.0015
final_learn_rate = 0.0005
init_mean = 0.01
standard_dev = .1;

steps_per_batch = np.ceil(train_dataset.shape[0]/batch_size)

loops_over_data = np.floor(num_steps/steps_per_batch)

# init_learn_rate*x**loops_over_data = 0.01
# x = (.01/init_learn_rate)**(1/loops_over_data)

decay_rate = (final_learn_rate/init_learn_rate)**(1/loops_over_data) # should be about .7 for 10 loops
print(decay_rate)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden],init_mean,standard_dev))
  biases = tf.Variable(tf.zeros([n_hidden]))

  weights1p5 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden2],init_mean,standard_dev))
  biases1p5 = tf.Variable(tf.zeros([n_hidden2]))
    
  weights2 = tf.Variable(tf.truncated_normal([n_hidden2, num_labels],init_mean,standard_dev))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
 
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(init_learn_rate, global_step, steps_per_batch, decay_rate, staircase=True)
  
    
    # Training computation.
  relu_layer_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases),.5)
  relu_layer_1p5_dropout = tf.nn.dropout(2*tf.nn.relu(tf.matmul(relu_layer_dropout,weights1p5)+biases1p5),.5)
  logits = tf.matmul(relu_layer_1p5_dropout, weights2) + biases2

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+L2_coef*tf.nn.l2_loss(weights)+L2_coef*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    
  relu_layer_NO_dropout = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
  relu_layer_1p5_NO_dropout = tf.nn.relu(tf.matmul(relu_layer_NO_dropout,weights1p5)+biases1p5)
  logits_NO_droput = tf.matmul(relu_layer_1p5_NO_dropout, weights2) + biases2
  train_prediction = tf.nn.softmax(logits_NO_droput)
    
  relu_layer_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights)+biases)
  relu_layer_1p5_valid = tf.nn.relu(tf.matmul(relu_layer_valid,weights1p5)+biases1p5)
  valid_prediction = tf.nn.softmax(tf.matmul(relu_layer_1p5_valid, weights2) + biases2)

  relu_layer_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights)+biases)
  relu_layer_1p5_test = tf.nn.relu(tf.matmul(relu_layer_test,weights1p5)+biases1p5)
  test_prediction = tf.nn.softmax(tf.matmul(relu_layer_1p5_test, weights2) + biases2)

0.924527663138


In [42]:

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    ##offset = (step % 2) * batch_size # this causes overfitting
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 55.534565
Minibatch accuracy: 8.2%
Validation accuracy: 10.7%
Minibatch loss at step 500: 13.986145
Minibatch accuracy: 73.8%
Validation accuracy: 71.1%
Minibatch loss at step 1000: 10.591033
Minibatch accuracy: 70.3%
Validation accuracy: 73.4%
Minibatch loss at step 1500: 5.281636
Minibatch accuracy: 77.3%
Validation accuracy: 74.3%
Minibatch loss at step 2000: 5.066291
Minibatch accuracy: 76.2%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 4.006958
Minibatch accuracy: 73.4%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 4.449730
Minibatch accuracy: 81.6%
Validation accuracy: 75.7%
Minibatch loss at step 3500: 3.904053
Minibatch accuracy: 70.3%
Validation accuracy: 75.5%
Minibatch loss at step 4000: 3.638338
Minibatch accuracy: 74.2%
Validation accuracy: 76.5%
Minibatch loss at step 4500: 4.050124
Minibatch accuracy: 68.8%
Validation accuracy: 75.2%
Minibatch loss at step 5000: 3.408622
Minibatch accuracy: 76.6%
Validation accur